In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

## score_set

In [ ]:
for n_element in (10,):

    element_score = np.random.normal(size=n_element // 2)

    element_score = pd.Series(
        np.sort(np.concatenate((element_score, -element_score))),
        name="Score",
        index=pd.Index(
            ("Element {}".format(i) for i in range(n_element)), name="Element"
        ),
    )

    for n_skip in (1,):

        if n_skip < 1:

            n_skip = int(n_skip * n_element)

        for n_top in (2,):

            if n_top < 1:

                n_top = int(n_top * n_element)

            for rate_top_hit in (1,):

                if rate_top_hit < 1:

                    rate_top_hit = int(rate_top_hit * n_top)

                for side in ("+",):

                    set_elements = []

                    if "-" in side:

                        set_elements += element_score.index[
                            n_skip : n_skip + n_top : rate_top_hit
                        ].tolist()

                    if "+" in side:

                        set_elements += element_score.index[
                            -(n_skip + 1) : -(n_skip + 1 + n_top) : -rate_top_hit
                        ].tolist()

                    for method in ("classic", "2020"):

                        kraft.score_set(
                            element_score,
                            set_elements,
                            method=method,
                            plot_process=True,
                        )

In [ ]:
element_score = pd.read_csv("data/gene_score.tsv", sep="\t", index_col=0, squeeze=True)

set_elements = kraft.read_gmt("data/c2.gmt")

In [ ]:
method = "2020"

In [ ]:
sets = []

sets += [
    "COLLER_MYC_TARGETS_UP",
]

sets += list(np.random.choice(list(set_elements.keys()), size=2, replace=False))

for set_ in sets:

    kraft.score_set(element_score, set_elements[set_], method=method, title=set_)

In [ ]:
set_score = {}

for set_, set_elements_ in set_elements.items():

    set_score[set_] = kraft.score_set(
        element_score, set_elements_, method=method, plot=False, title=set_
    )

set_score = pd.Series(set_score).sort_values()

n_extreme = 2

for i in list(range(n_extreme)) + list(range(-n_extreme, 0)):

    set_ = set_score.index[i]

    kraft.score_set(element_score, set_elements[set_], method=method, title=set_)